In [6]:
import math
from mpmath import cos, acos, acot, tan
import numpy as np
import cirq
import time
from math import pi, floor, ceil, log, cos, sin,sqrt
import sys

In [7]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [10]:
def mcx(q, index, tgt):
    if len(index) == 0:
        yield cirq.X(q[tgt])
    elif len(index) == 1:
        yield cirq.CNOT(q[index[0]], q[tgt])
    elif len(index) == 2:
        yield cirq.CCX(q[index[0]], q[index[1]], q[tgt])
    else:
        yield cirq.H(q[tgt])
        yield mcx(q,index[:-1], tgt)
        yield cirq.T**-1(q[tgt])
        yield cirq.CNOT(q[index[len(index)-1]], q[tgt])
        yield cirq.T(q[tgt])
        yield mcx(q,index[:-1], tgt)
        yield cirq.T**-1(q[tgt])
        yield cirq.CNOT(q[index[len(index)-1]], q[tgt])
        yield cirq.T(q[tgt])
        yield cirq.H(q[tgt])
        yield incrementer(q, index)
        for i, j in reversed(index[1:]), range(len(index)):
            yield cirq.ZPowGate(-0.5/(pow(2, j+2)), q[i])
        yield decrementer(q, index)
        for i, j in reversed(index[1:]), range(len(index)):
            yield cirq.ZPowGate(0.5/(pow(2, j+2)), q[i])
        yield cirq.u1(pi/(pow(2, len(index))), q[index[0]])
        
def incrementer(q, index):
    for i in range(len(index)-1,1,-1):
        yield mcx(q, index[:-1], index[len(index)-1])
    if len(index) > 1:
        yield cirq.CNOT(q[int(index[0])],q[int(index[1])])
    yield cirq.X(q[int(index[0])])

def decrementer(q, index):
    yield cirq.X(q[int(index[0])])
    if len(index) > 1:
        yield cirq.CNOT(q[int(index[0])],q[int(index[1])])
    for i in range(2,len(index)):
        yield mcx(q, index[:i], index[i])

def c_incrementer(q, index, controls):
    for i in range(len(controls)):
        coerced = np.concatenate([index, controls[:len(controls)-i-1]])
        coerced = [int(i) for i in coerced]
        yield mcx(q, coerced, int(controls[len(controls)-i-1]))

def c_decrementer(q, index, controls):
    for i in range(len(controls)):
        coerced = np.concatenate([index, controls[:-len(controls)+i]])
        coerced = [int(i) for i in coerced]
        yield mcx(q, coerced, int(controls[i]))

In [25]:
def mcu1(q, index, tgt, theta):
    theta = theta/(2*pi)
    if len(index) == 0:
        yield cirq.ZPowGate(theta)(q[tgt])
    elif len(index) == 1:
        yield cirq.CZPowGate(theta)(q[index[0]], q[tgt])
    else:
        yield incrementer(q, index)
        for i, j in reversed(index[1:]), range(len(index)):
            yield cirq.ZPowGate(-theta/(pow(2, j+1)))(q[i])
        yield decrementer(q, index)
        for i, j in reversed(index[1:]), range(len(index)):
            yield cirq.ZPowGate(theta/(pow(2, j+1)))(q[i])
        yield cirq.ZPowGate(theta/(pow(2, len(index))))(q[index[0]])

In [27]:
def mean_inversion_cnf(q, n):
    ## Hadamard gate
    for i in range(0, n):
        yield cirq.H(q[i])
    ## Pauli X gate
    for i in range(n):
        yield cirq.X(q[i])
    yield mcx(q, range(n), n)
    ## Pauli X gate
    for i in range(n):
        yield cirq.X(q[i])
    ## Hadamard gate
    for i in range(n):
        yield cirq.H(q[i])

In [28]:
def mean_inversion_fpaa(q, n):
    yield cirq.X(q[n+1])
    yield cirq.H(q[n+1])
    ## Hadamard gate
    for i in range(0, n):
        yield cirq.H(q[i])
    ## Pauli X gate
    for i in range(n):
        yield cirq.X(q[i])
    yield mcx(q, range(n), n+1)
    ## Pauli X gate
    for i in range(n):
        yield cirq.X(q[i])
    ## Hadamard gate
    for i in range(n):
        yield cirq.H(q[i])
    yield cirq.H(q[n+1])
    yield cirq.X(q[n+1])

In [ ]:
def cnf_oracle(q, n, a, oracle, index):
    b = floor(log(a,2))+1
    temp = [int(x) for x in np.binary_repr(a, width=b)]
    incr = []
    for i in range(b):
        incr.append(n+i+1)
    for j in range(a):
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])
        yield c_incrementer(q, index[j],n+1+np.array(range(b)))
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])
    for i in range(b):
        yield cirq.X(q[n+1+i])
    yield mcx(q, incr, n)
    for i in range(b):
        yield cirq.X(q[n+1+i])
    for j in range(a-1, -1, -1):
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])
        yield c_decrementer(q, index[j],n+1+np.array(range(b)))
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])

In [29]:
def fpaa_oracle(q, n, clauses, oracle, index):
    theta = pi/clauses
    delta = pow(2,-0.5*pow(n,2))/2
    _lambda = pow(sin(theta),2)/4+pow(1/2-cos(theta)/2,2)
    L = int(ceil(2*log(2/delta)/sqrt(_lambda)))
    gamma = cos(acos(1/delta)/L)
    
    yield A_matrix(q, n, clauses, oracle, theta)
    for k in range(1,L):
        alpha = abs(2*acot(tan(2*pi*(  k   )/L)*mpsqrt(1-1/pow(gamma,-2))))
        beta = -abs(2*acot(tan(2*pi*(L-(k-1))/L)*mpsqrt(1-1/pow(gamma,-2))))
        yield cirq.H(q[n])
        yield cirq.ZPowGate(beta/(2*pi))(q[n],q[n+1])
        yield cirq.H(q[n])
        yield Adgr_matrix(q, n, clauses, oracle, theta)
        yield cirq.H(q[n])
        yield cirq.ZPowGate(alpha/(2*pi))(q[n],q[n+1])
        yield cirq.H(q[n])
        yield A_matrix(q, n, clauses, oracle, theta)
    yield cirq.H(q[n])
    
    yield cirq.X(q[n])
    yield cirq.X(q[n+1])
    yield cirq.H(q[n+1])
    yield cirq.CNOT(q[n],q[n+1])
    yield cirq.H(q[n+1])
    yield cirq.X(q[n+1])
    yield cirq.X(q[n])

def A_matrix(q, n, clauses, oracle, theta):
    for j in range(clauses):
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])
        yield mcu1(q, range(n+1), n+1, theta)
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])

def Adgr_matrix(q, n, clauses, oracle, theta):
    for j in range(clauses):
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])
        yield mcu1(q, range(n+1), n+1, -theta)
        for i in range(n):
            if not abs(oracle[j][i]):
                yield cirq.X(q[i])

In [31]:
def grover_search_cnf(n, clauses, shots, oracle, index, nr_solutions):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = floor((pi*sqrt(pow(2, n)/nr_solutions))/4)
    a = floor(log(clauses,2)+1)

    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if (n+1) + a > 24:
        print("Too much qubits needed! (", (n+1) + a,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = [cirq.LineQubit(i) for i in range(n+1+a)]
    key = ""
    for i in range(n+1+a):
        key += str(i)
    c = cirq.Circuit()
    
    for i in range(n):
        yield cirq.H(q[i])
    yield cirq.X(q[n])
    yield cirq.H(q[n])

    for i in range(iterations):
        c.append(cnf_oracle(q, n, clauses, oracle, index))
        c.append(mean_inversion_cnf(q, n))

        # for i in range(n,n+a):
        #     yield cirq.measure(q[i], c[0])
        #     yield cirq.X(q[i]).c_if(c,1)

    for i in range(n):
        c.append(cirq.measure(q[i], c[i]))

    simulator = cirq.Simulator()
    result = simulator.run(c, repetitions=shots)
    counter = (results.multi_measurement_histogram(keys=key))

    print(result().get_counts())
    # Draw the circuit
    print(c)

In [ ]:
def grover_search_fpaa(n, shots, oracle, index, nr_solutions):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = floor((pi*sqrt(pow(2, n)/nr_solutions))/4)
    
    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if n+2 > 24:
        print("Too much qubits needed! (", n+2,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister(n + 2 )
    c = ClassicalRegister(n + 2 )
    yield cirq = QuantumCircuit(q, c)

    for i in range(floor(n/2)):
        yield cirq.SWAP(q[i],q[n-i-1])
        
    for i in range(n):
        yield cirq.H(q[i])
        
    for i in range(iterations):
        fpaa_oracle(q, n, clauses, oracle, index)
        mean_inversion_fpaa(q, n)

        # for i in range(n,n+a):
        #     yield cirq.measure(q[i], c[0])
        #     yield cirq.X(q[i]).c_if(c,1)

    for i in range(floor(n/2)):
        yield cirq.SWAP(q[i],q[n-i-1])
        
    for i in range(n):
        yield cirq.measure(q[i], c[i])

    backend = qk.BasicAer.get_backend('qasm_simulator')
    job = execute(backend=backend, shots=shots, max_credits=3)
    print(job.result().get_counts())
    # Draw the circuit
    print(yield cirq)

In [ ]:
def make_cnf():
    # 1 = variable, 0 = negated variable, -1 = not in clause
    cnf = [
        [0, 0, -1],
        [0, 1, -1],
        [1, 0, -1],
        [1, -1, 0]
    ]
    # values are the indexes of each variable in the clause
    index = [
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 2]
    ]
    # number of variables
    n = 3
    # number of clauses
    clauses = 4
    # number of solutions
    nr_solutions = 1
    return n, clauses, cnf, index, nr_solutions

In [ ]:
n, clauses, oracle, index, nr_solutions = make_cnf()
shots = 100

grover_search_cnf(n, clauses, shots, oracle, index, nr_solutions)
# grover_search_fpaa(n, shots, oracle, index, nr_solutions)

In [ ]:
def grover_search_cnf_noisy(n, clauses, shots, oracle, index, nr_solutions, value):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = floor((pi*sqrt(pow(2, n)/nr_solutions))/4)
    a = floor(log(clauses,2)+1)

    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if (n+1) + a > 24:
        print("Too much qubits needed! (", (n+1) + a,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister(n +1 + a )
    c = ClassicalRegister(n + 1 + a )
    yield cirq = QuantumCircuit(q, c)
    
    for i in range(n):
        yield cirq.H(q[i])
    yield cirq.X(q[n])
    yield cirq.H(q[n])

    for i in range(iterations):
        cnf_oracle(q, n, clauses, oracle, index)
        mean_inversion_cnf(q, n)

        for i in range(n,n+a):
            yield cirq.measure(q[i], c[0])
            yield cirq.X(q[i]).c_if(c,1)

    for i in range(n):
        yield cirq.measure(q[i], c[i])


    backend = Aer.get_backend('qasm_simulator')

    # Execute noisy simulation and get counts
    device = IBMQ.get_provider().get_backend('ibmq_london')
    properties = device.properties()
    coupling_map = device.configuration().coupling_map

    for _, qubit_props in enumerate(properties.qubits):
        for item in qubit_props:
            if item.name == 'T1':
                item.value = value
            if item.name == 'T2':
                item.value = value
    for gate in properties.gates:
        for item in gate.parameters:
            if item.name == 'gate_error':
                item.value = 0.00000001
    gate_times = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200), ('cx', None, 0)
    ]
    # Construct the noise model from backend properties
    # and custom gate times
    noise_model = noise.device.basic_device_noise_model(properties,readout_error=False, thermal_relaxation=True, gate_times=gate_times)

    # Get the basis gates for the noise model
    basis_gates = noise_model.basis_gates

    result = execute(backend=backend, shots=100, noise_model=noise_model, basis_gates=basis_gates).result()
    counts = result.get_counts(yield cirq)
    return counts

In [ ]:
def grover_search_fpaa_noisy(n, shots, oracle, index, nr_solutions, value):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = floor((pi*sqrt(pow(2, n)/nr_solutions))/4)
    
    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if n+2 > 24:
        print("Too much qubits needed! (", n+2,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister(n + 2 )
    c = ClassicalRegister(n + 2 )
    yield cirq = QuantumCircuit(q, c)

    for i in range(floor(n/2)):
        yield cirq.SWAP(q[i],q[n-i-1])
        
    for i in range(n):
        yield cirq.H(q[i])
        
    for i in range(iterations):
        fpaa_oracle(q, n, clauses, oracle, index)
        mean_inversion_fpaa(q, n)

        for i in range(n,n+2):
            yield cirq.measure(q[i], c[0])
            yield cirq.X(q[i]).c_if(c,1)

    for i in range(floor(n/2)):
        yield cirq.SWAP(q[i],q[n-i-1])
        
    for i in range(n):
        yield cirq.measure(q[i], c[i])

    backend = Aer.get_backend('qasm_simulator')

    # Execute noisy simulation and get counts
    device = IBMQ.get_provider().get_backend('ibmq_london')
    properties = device.properties()
    coupling_map = device.configuration().coupling_map

    for _, qubit_props in enumerate(properties.qubits):
        for item in qubit_props:
            if item.name == 'T1':
                item.value = value
            if item.name == 'T2':
                item.value = value
    for gate in properties.gates:
        for item in gate.parameters:
            if item.name == 'gate_error':
                item.value = 0.00000001
    gate_times = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200), ('cx', None, 0)
    ]
    # Construct the noise model from backend properties
    # and custom gate times
    noise_model = noise.device.basic_device_noise_model(properties,readout_error=False, thermal_relaxation=True, gate_times=gate_times)

    # Get the basis gates for the noise model
    basis_gates = noise_model.basis_gates

    result = execute(backend=backend, shots=100, noise_model=noise_model, basis_gates=basis_gates).result()
    counts = result.get_counts(yield cirq)
    return counts

In [ ]:
start_val = 1
end_val   = 100
runs = 10
shots = 100
n, clauses, oracle, index, nr_solutions = make_cnf()

dist = end_val-start_val+(end_val - start_val) / (runs-1)
values = np.arange(start_val,end_val+(end_val - start_val) / (runs-1), dist/runs)
data = []

printProgressBar(0, len(values), prefix = 'Progress:', suffix = 'Complete', length = len(values))
for i in range(len(values)):
    data.append(grover_search_cnf_noisy(n, clauses, shots, oracle, index, nr_solutions, values[i]))
#     data.append(grover_search_fpaa_noisy(n, shots, oracle, index, nr_solutions, values[i]))
    printProgressBar(i, len(values), prefix = 'Progress:', suffix = 'Complete', length = len(values))

print(data)